In [11]:
import sys
import os
import json
import ast
from collections import deque, defaultdict
sys.path.insert(0, os.path.abspath('..'))
from cube_simulator_for_table_generators import Cube

In [12]:
def generate_move_table():
    """
    Generates a table mapping each move to the positions it affects and their new positions.

    Returns:
        dict: A dictionary where keys are move names and values are dictionaries
              mapping initial positions to final positions after the move.
    """
    # Create a fresh Cube object
    cube = Cube()
    movement_table = {}

    tracked_positions = [(i,j,k) for i in range(0,3) for j in range(0,3) for k in range(0,3)]

    movement_table = defaultdict(dict)
    for move in cube.move_map.keys():
        # Get a fresh cube for each move calculation
        test_cube = Cube()

        # Apply the move
        test_cube.apply_moves(move)
        for initial_pos in tracked_positions:
            piece_id_to_track = test_cube.piece_initial_ids_at_positions[initial_pos]
            final_pos = test_cube.get_position_of_piece(piece_id_to_track) # Find where that piece ended up
            movement_table[move][initial_pos] = final_pos

        del test_cube

    # Save to file
    with open('../Precomputed_Lookup_Tables/movement_table.json', 'w') as f:
        # Convert tuple positions to strings for JSON serialization
        serializable_table = {}
        for move, position_movements in movement_table.items():
            serializable_movements = {}
            for from_pos, to_pos in position_movements.items():
                from_pos_str = str(from_pos)
                to_pos_str = str(to_pos)
                serializable_movements[from_pos_str] = to_pos_str
            serializable_table[move] = serializable_movements

        json.dump(serializable_table, f, indent=2)
    
    #Acknowledge successful table creation
    print("Created the table Successfully")

In [13]:
generate_move_table()

Created the table Successfully


In [14]:
def generate_distance_table(piece_type:str, filename:str):
    """
    Calculates the minimum distances (ignoring the piece orientations and the rest of the cube) between each pair of the pieces of the given piece type, using Breadth-First Search (BFS)
    """

    # Load the position_movement_table created in the previous cell:
    try:
        filename_1 = "../Precomputed_Lookup_Tables/movement_table.json"
        with open(filename_1, 'r') as f:
            serializable_table:dict = json.load(f)
    except Exception as e:
        print(f"Failed to load {filename_1}: {e}")

    # Serialize the loaded position_movement json table with the structure -
    # {position(tuple): {move(str): new_position(tuple)}}
    movement_table = defaultdict(dict)
    for move, position_movements in serializable_table.items():
        for from_pos_str, to_pos_str in position_movements.items():
            from_pos = tuple(ast.literal_eval(from_pos_str))
            to_pos = tuple(ast.literal_eval(to_pos_str))
            movement_table[from_pos][move] = to_pos

    solved_cube = Cube()
    all_moves = list(solved_cube.move_map.keys())
    valid_positions = solved_cube.edge_positions if piece_type == "edge" else solved_cube.corner_positions

    # Algorithm starts here
    distance_table = {}
    for start_pos in valid_positions:
        for target_pos in valid_positions:
            if start_pos == target_pos:
                distance_table[(start_pos, target_pos)] = 0
                continue

            if (target_pos, start_pos) in distance_table: #symmetry optimization
                distance_table[(start_pos, target_pos)] = distance_table[(target_pos, start_pos)]
                continue

            # "Breadth-First Search" (BFS) begins
            visited = set([start_pos])
            queue = deque([(start_pos, 0)])
            found_distance = -1

            while queue:
                current_pos, dist = queue.popleft()
                if current_pos == target_pos:
                    found_distance = dist
                    break
                
                # Explore all possible moves from current_pos in the table/graph
                for move in movement_table[current_pos]:
                    next_pos = movement_table[current_pos][move]
                    if next_pos not in visited:
                        visited.add(next_pos)
                        queue.append((next_pos, dist + 1))

            distance_table[(start_pos, target_pos)] = found_distance

    serializable_table = {}
    for pos_pair, dist in distance_table.items(): # Key is position pair
        serializable_table[str(pos_pair)] = dist
    if piece_type not in filename or "distance" not in filename:
        raise ValueError("Incorrect filename")
    with open(filename, 'w') as f:
        json.dump(serializable_table, f, indent=2)

    # Acknowledge successful table creation
    print("Created the table Successfully")

In [15]:
generate_distance_table("edge", "../Precomputed_Lookup_Tables/edge_primary_distance_table.json")
generate_distance_table("corner", "../Precomputed_Lookup_Tables/corner_primary_distance_table.json")

Created the table Successfully
Created the table Successfully
